In [8]:
"""
Temporary routine for generation of datasets
for demonstration purposes

It is a "Fake Data Mediator" for use with the first
version of the Interactive Topic Model Trainer
"""

import argparse
import json
import os
from pathlib import Path
from langdetect import detect
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

In [2]:
#This variables will be passed as parameters via command line

#parquet_table = '/export/ml4ds/IntelComp/Datalake/SemanticScholar/20220201/papers.parquet'
#selectFields = 'title, paperAbstract, doi, year, fieldsOfStudy'
#filterCondition = "array_contains(fieldsOfStudy, 'Computer Science')"
#path_dataset = "/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/fromHDFS/S2CS"

parquet_table = '/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects.parquet'
selectFields = 'title, objective, startDate, ecMaxContribution, euroSciVocCode'
filterCondition = ''
path_dataset = "/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/fromHDFS/CORDIS"

In [3]:
# We read the table with the output of NLP processes and identify id field 
lemmas_table = parquet_table.split('.parquet')[0] + '_NLP.parquet'
lemmas_df = spark.sql(f"SELECT * FROM parquet.`{lemmas_table}`")
# Obtain the name of the column that has to be used as the main identifier for the corpus
id_fld = [el for el in lemmas_df.columns if el in ['projectID', 'id', 'appln_id']][0]

22/05/21 19:12:02 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/05/21 19:12:02 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/05/21 19:12:04 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/05/21 19:12:04 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jarenas@192.168.148.225
22/05/21 19:12:05 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/05/21 19:12:05 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [4]:
# We read the main table including selected fields and the identifier
flds = [el.strip() for el in selectFields.split(',')]
query = "SELECT " + id_fld + " AS id, " + (",").join(flds) + \
                " FROM parquet.`" + parquet_table + "`"

# Add filtering condition to SELECT clause if necessary
if len(filterCondition.strip()):
    query += " WHERE " + filterCondition
dataset = spark.sql(query)

22/05/21 19:12:18 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [6]:
# Join tables
lemmas_df = lemmas_df.withColumnRenamed(id_fld,"id")
dataset = (dataset.join(lemmas_df, dataset.id ==  lemmas_df.id, "left")
                      .drop(lemmas_df.id)
                )

print('Number of documents in dataset:', dataset.count())
#dataset.show(n=10, truncate=120, vertical=True)

Number of documents in dataset: 61163


In [7]:
# Save dataset
dataset.write.parquet(f"file://{path_dataset}",
    mode="overwrite",
)